In [28]:
import math
import os

import numpy as np
from keras import Sequential, Model
from keras.layers import Dense
from tensorflow.python.keras.optimizer_v2.adam import Adam

from src.data.preprocess.hw import hamming_weights
from src.dlla.hw import encode, dlla_hw
from src.dlla.wegener import make_mlp_wegener, binomial_test
from src.pollution.clock_jitter import clock_jitter
from src.pollution.gaussian_noise import gaussian_noise
from src.pollution.random_delay import random_delay
from src.trace_set.database import Database
from src.trace_set.pollution import Pollution, PollutionType
from src.trace_set.set_hw import TraceSetHW
from src.trace_set.transform import reduce_fixed_fixed, fixed_fixed
from src.trace_set.window import get_windows
from src.tvla.cri import tvla_cri
from src.tvla.tvla import Group

In [29]:
DB = Database.aisy

In [30]:
RAW_TRACES = TraceSetHW(DB)

X_PROF_CXT, Y_PROF = RAW_TRACES.profile_states()
X_ATT_CXT, Y_ATT = RAW_TRACES.attack_states()

A_THIRD = round(X_PROF_CXT.shape[1] / 3)

WINDOW_START = 0
WINDOW_END = 300
SAMPLE_TRACE = X_PROF_CXT[0][WINDOW_START:WINDOW_END]

In [31]:
X_PROF = X_PROF_CXT[:, 0:300]
X_ATT = X_ATT_CXT[:, 0:300]

X_PROF_CXT.shape, X_PROF.shape, Y_PROF.shape

X_PROF.shape

(6667, 300)

In [49]:
GAUSS_PARAMS = np.arange(0, 82, 2)

def apply_gauss(params):
    for param in params:
        pollution = Pollution(PollutionType.gauss, param)
        out = TraceSetHW(DB, pollution)

        if not os.path.exists(out.path):
            x = gaussian_noise(X_PROF, param)
            x_att = gaussian_noise(X_ATT, param)

            out.create(x, Y_PROF, x_att, Y_ATT)

            x2, y2 = fixed_fixed(x, hamming_weights(Y_PROF))
            a, b = x2[~y2], x2[y2]
            order = 2
            _, tvla_p = Group(a, order, True).t_test(Group(b, order, True), order)
            print(f"Gaussian noise ({param}): min-p: ({min(tvla_p)}).")

apply_gauss(GAUSS_PARAMS)

Computing Central Moments: 100%|██████████| 4/4 [00:00<00:00, 456.93it/s]


Gaussian noise (42): min-p: (0.01429307144138616).


Computing Central Moments: 100%|██████████| 4/4 [00:00<00:00, 470.38it/s]


Gaussian noise (44): min-p: (0.0013228298569443713).


Computing Central Moments: 100%|██████████| 4/4 [00:00<00:00, 430.97it/s]


...

Gaussian noise (76): min-p: (0.008666757841560969).


Computing Central Moments: 100%|██████████| 4/4 [00:00<00:00, 476.72it/s]


Gaussian noise (78): min-p: (0.001785572250472735).


Computing Central Moments: 100%|██████████| 4/4 [00:00<00:00, 475.22it/s]

Gaussian noise (80): min-p: (0.006875354885335225).


### TVLA vs. DL-LA

In [33]:
ORDER = 2

def store_results(database: Database, method: str, pollution: Pollution, p):
    file_name = f"results_{database.name}.csv"
    with open(file_name, 'a') as f:
        f.write(f"{method};{pollution.type.name};{pollution.parameter};{p}\n")

In [53]:
def prepare_traces_dl(x, y, x_att, y_att):
    """
    Normalizes the traces, one-hot encodes the labels.
    Returns profiling traces, labels and attack traces, labels.
    """
    prof_mean, prof_std = x.mean(axis=0), x.std(axis=0)
    norm_x = (x - prof_mean) / prof_std
    norm_x_att = (x_att - prof_mean) / prof_std

    return norm_x, encode(y), norm_x_att, encode(y_att)


def build_mlp(x, y, params):
    mdl = Sequential()
    mdl.add(Dense(100, activation=params['activation'], input_shape=(x.shape[1],)))
    mdl.add(Dense(100, activation=params['activation']))
    mdl.add(Dense(100, activation=params['activation']))
    mdl.add(Dense(100, activation=params['activation']))
    mdl.add(Dense(9, activation='softmax'))

    mdl.compile(optimizer=params['optimizer'], loss=params['losses'], metrics=['accuracy'])

    out = mdl.fit(x, y, shuffle=True, batch_size=params['batch_size'], epochs=params['epochs'], verbose=False)

    return out, mdl


def make_mlp(x, y):
    return build_mlp(x, y, {
        'activation': 'relu',
        'optimizer': Adam(learning_rate=0.001),
        'losses': 'categorical_crossentropy',
        'batch_size': 150,
        'epochs': 5
    })[1]

def wegener_p(mdl: Model, x_att: np.ndarray, y_att: np.ndarray):
    predictions = mdl.predict(x_att).argmax(axis=1)
    labels = y_att.argmax(axis=1)

    correct = np.sum(predictions == labels)
    total = len(predictions)
    # print(total, correct)

    return binomial_test(total, correct)

LIMIT_PROF, LIMIT_ATT = None, 100000

def la_benchmark(db: Database, pollution_type: PollutionType, params):
    for param in params:
        pollution = Pollution(pollution_type, param)
        print("Load traces         ", end="\r")
        trace_set = TraceSetHW(db, pollution, (LIMIT_PROF, LIMIT_ATT))

        if os.path.exists(trace_set.path):
            print("Prepare traces (1/2)", end="\r")
            x9, y9, x9_att, y9_att = prepare_traces_dl(*trace_set.profile(), *trace_set.attack())
            print("Prepare traces (2/2)", end="\r")
            (x2, y2), (x2_att, y2_att) = reduce_fixed_fixed(x9, y9), reduce_fixed_fixed(x9_att, y9_att)

            print("Make model (1/2)    ", end="\r")
            mdl9 = make_mlp(x9, y9)
            print("Make model (2/2)    ", end="\r")
            mdl2 = make_mlp_wegener(x2, y2, False)

            print("TVLA                ", end="\r")
            tvla_ps = np.min(tvla_cri(*fixed_fixed(*trace_set.profile()), ORDER), axis=1)

            print("Predict             ", end="\r")
            dlla9_p = dlla_hw(mdl9, x9_att, y9_att)
            dlla2_p = wegener_p(mdl2, x2_att, y2_att)

            print(f"{pollution_type} ({param}). TVLA ({tvla_ps}). DLLA9 ({dlla9_p}). DLLA2 ({dlla2_p})", end="\r")

            for order, p in enumerate(tvla_ps):
                if order > 0:
                    store_results(db, f"cri_tvla_{order}", pollution, p)

            store_results(db, "dlla9", pollution, dlla9_p)
            store_results(db, "dlla2", pollution, dlla2_p)

        print()

while True:
    la_benchmark(DB, PollutionType.gauss, GAUSS_PARAMS)


PollutionType.gauss (0). TVLA ([1.00000000e+00 1.31293959e-54 3.48941128e-09]). DLLA9 (2.7525710085775305e-169). DLLA2 (1.426062235324097e-49)
PollutionType.gauss (2). TVLA ([1.00000000e+00 3.26820702e-51 1.24499572e-08]). DLLA9 (1.750034334239383e-158). DLLA2 (2.253385540775225e-56)
PollutionType.gauss (4). TVLA ([1.00000000e+00 1.22228456e-45 1.48403364e-07]). DLLA9 (6.367590722810699e-135). DLLA2 (4.755946889135307e-47)
PollutionType.gauss (6). TVLA ([1.00000000e+00 1.78265312e-39 3.13618890e-06]). DLLA9 (5.602140633529632e-124). DLLA2 (6.875909982109351e-40)
PollutionType.gauss (8). TVLA ([1.00000000e+00 2.87463913e-31 1.35494969e-04]). DLLA9 (7.352753533625524e-89). DLLA2 (4.791963372743627e-33)
PollutionType.gauss (10). TVLA ([1.00000000e+00 1.95921301e-24 3.48932512e-04]). DLLA9 (1.5959163447361152e-87). DLLA2 (1.1204926917619186e-27)
PollutionType.gauss (12). TVLA ([1.00000000e+00 5.45178693e-21 8.09581204e-03]). DLLA9 (3.5129329788481994e-66). DLLA2 (7.561519562372346e-22)
Pol

KeyboardInterrupt: 